In [1]:
import os
import cv2

In [2]:
base_dir = r"phone_defect_detect"

sub_dirs = ["defect_phone", "good_phone", "New_phone"]


### Open all of the Images folders

In [3]:
#pip install --user opencv-contrib-python

for sub_dir in sub_dirs:
    folder_path = os.path.join(base_dir, sub_dir)
    if os.path.exists(folder_path):
        print(f"\nOpening images in folder: {sub_dir}")
        files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]
        for file_name in files:
            file_path = os.path.join(folder_path, file_name)
            print(f"Displaying: {file_name}")
            image = cv2.imread(file_path)
            cv2.imshow(f"{sub_dir} - {file_name}", image)
            key = cv2.waitKey(0)
            if key == 27:
                break
        cv2.destroyAllWindows()
    else:
        print(f"Folder {sub_dir} does not exist.")


Opening images in folder: defect_phone
Displaying: HuaweiNovaY72.jpg

Opening images in folder: good_phone
Displaying: good.jpg

Opening images in folder: New_phone


### Crate output folder for Processing Images

In [ ]:
output_base_dir = os.path.join(base_dir, "processed_img")
os.makedirs(output_base_dir, exist_ok=True)

### Define Each preprocessing techniques

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, threshold1= 50, threshold2=150)
    return edges

def apply_clahe(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    return enhanced

def denoise_image(image):
    denoised = cv2.fastNlMeansDenoisingColored(image, None, h=10, hForColor=10, templateWindowSize=7)
    return denoised

def apply_fourier_transform(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum = 20 * np.log(np.abs(fshift))
    return magnitude_spectrum


In [ ]:

for sub_dir in sub_dirs:
    folder_path = os.path.join(base_dir, sub_dir)
    output_dir = os.path.join(output_base_dir, sub_dir)
    os.makedirs(output_dir, exist_ok=True)

    if os.path.exists(folder_path):
        print(f"\nProcessing images in folder: {sub_dir}")
        files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

        for file_name in files:
            file_path = os.path.join(folder_path, file_name)
            print(f"Processing: {file_name}")
            image = cv2.imread(file_path)

            resized = cv2.resize(image, (224, 224))
            edges = edge_detection(image)
            clahe_image = apply_clahe(image)
            denoised = denoise_image(image)
            fourier = apply_fourier_transform(image)

            cv2.imwrite(os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_resized.jpg"), resized)
            cv2.imwrite(os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_edges.jpg"), edges)
            cv2.imwrite(os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_clahe.jpg"), clahe_image)
            cv2.imwrite(os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_denoised.jpg"), denoised)
            plt.imsave(os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_fourier.jpg"), fourier, cmap='gray')

            plt.figure(figsize=(10, 6))
            plt.subplot(2, 3, 1)
            plt.imshow(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
            plt.title("Resized")
            plt.axis("off")

            plt.subplot(2, 3, 2)
            plt.imshow(edges, cmap="gray")
            plt.title("Edge Detection")
            plt.axis("off")

            plt.subplot(2, 3, 3)
            plt.imshow(clahe_image, cmap='gray')
            plt.title("CLAHE")
            plt.axis("off")

            plt.subplot(2, 3, 4)
            plt.imshow(cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY))
            plt.title("Denoised")
            plt.axis("off")

            plt.subplot(2, 3, 5)
            plt.imshow(fourier, cmap='gray')
            plt.title("Fourier Transform")
            plt.axis("off")

            plt.tight_layout()
            plt.show()

else:
    print(f"Folder {sub_dir} does not exist.")